In [37]:
from tensorflow import keras
import database_connect as db_connect
import pandas as pd

import numpy as np
import plotting

In [38]:
CONFIG = db_connect.load_config()

In [39]:
def dataframe_to_tensor(df, timestep_length: int):
    
    '''
    Converts a pandas dataframe to a tensor
    
    Input: Pandas dataframe
            timestep_length (int) - the length of the timestep
    Output: Tensor
    
    '''
    
    # Convert the dataframe to a numpy array
    df = np.array(df)

    n_seq = len(df) - timestep_length + 1
    return np.array([df[i:(i+timestep_length)] for i in range(n_seq)])

In [40]:
columns = ['id',
    'hour_sin',
 'hour_cos',
 'day_sin',
 'day_cos',
 'mon_sin',
 'mon_cos',
 'weekday_sin',
 'weekday_cos',
 'year',
 'RATIO_close_and_MA3',
 'RATIO_close_and_MA6',
 'RATIO_close_and_MA12',
 'RATIO_close_and_MA24',
 'RATIO_close_and_MA48',
 'RATIO_close_and_MA96',
 'RATIO_close_and_MA192',
 'RATIO_close_and_MA384',
 'RATIO_quote_asset_volume_and_MA3',
 'RATIO_quote_asset_volume_and_MA6',
 'RATIO_quote_asset_volume_and_MA12',
 'RATIO_quote_asset_volume_and_MA24',
 'RATIO_quote_asset_volume_and_MA48',
 'RATIO_quote_asset_volume_and_MA96',
 'RATIO_quote_asset_volume_and_MA192',
 'RATIO_quote_asset_volume_and_MA384',
 'RATIO_num_trades_and_MA3',
 'RATIO_num_trades_and_MA6',
 'RATIO_num_trades_and_MA12',
 'RATIO_num_trades_and_MA24',
 'RATIO_num_trades_and_MA48',
 'RATIO_num_trades_and_MA96',
 'RATIO_num_trades_and_MA192',
 'RATIO_num_trades_and_MA384',
 'RATIO_close_and_STD3',
 'RATIO_close_and_STD6',
 'RATIO_close_and_STD12',
 'RATIO_close_and_STD24',
 'RATIO_close_and_STD48',
 'RATIO_close_and_STD96',
 'RATIO_close_and_STD192',
 'RATIO_close_and_STD384',
 'RSI_3_close',
 'RSI_6_close',
 'RSI_12_close',
 'RSI_24_close',
 'RSI_48_close',
 'RSI_96_close',
 'RSI_192_close',
 'RSI_384_close',
 'RSI_3_quote_asset_volume',
 'RSI_6_quote_asset_volume',
    'RSI_12_quote_asset_volume', 'RSI_24_quote_asset_volume',
    'RSI_48_quote_asset_volume', 'RSI_96_quote_asset_volume',
    'RSI_192_quote_asset_volume', 'RSI_384_quote_asset_volume',
    'RSI_3_num_trades', 'RSI_6_num_trades', 'RSI_12_num_trades',
    'RSI_24_num_trades', 'RSI_48_num_trades', 'RSI_96_num_trades',
    'RSI_192_num_trades', 'RSI_384_num_trades' ,
 'predicted']

In [73]:
model = keras.models.load_model('models/2024-02-12 00-52-43 val_PR 0.682 Dense.hd5', compile=False)

In [74]:
TIME_STEP = 12

In [75]:
model.summary()

Model: "model_34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 65)]              0         
                                                                 
 normalization_34 (Normaliz  (None, 65)                131       
 ation)                                                          
                                                                 
 dense_93 (Dense)            (None, 325)               21450     
                                                                 
 dense_94 (Dense)            (None, 260)               84760     
                                                                 
 dense_95 (Dense)            (None, 195)               50895     
                                                                 
 output (Dense)              (None, 4)                 784       
                                                          

In [76]:
conn = db_connect.connect(CONFIG)

Connected to the PostgreSQL server.


In [77]:
# sql = 'select min(timestamp) from trading_bot_historydataeth \
#         where predicted is null and hour_sin is not null \
#         limit 1'

sql = 'select min(timestamp) from trading_bot_historydataeth \
        where hour_sin is not null \
        limit 1'


In [78]:
with conn.cursor() as cursor:
    cursor.execute(sql)
    result = cursor.fetchone()

In [79]:
result = result[0]

In [80]:
result_str = result.strftime('%Y-%m-%d %H:%M:%S')
result_str

'2024-01-09 06:00:00'

In [81]:
sql = 'SELECT "'
sql += '", "'.join(columns)
sql += '"'

In [82]:
sql

'SELECT "id", "hour_sin", "hour_cos", "day_sin", "day_cos", "mon_sin", "mon_cos", "weekday_sin", "weekday_cos", "year", "RATIO_close_and_MA3", "RATIO_close_and_MA6", "RATIO_close_and_MA12", "RATIO_close_and_MA24", "RATIO_close_and_MA48", "RATIO_close_and_MA96", "RATIO_close_and_MA192", "RATIO_close_and_MA384", "RATIO_quote_asset_volume_and_MA3", "RATIO_quote_asset_volume_and_MA6", "RATIO_quote_asset_volume_and_MA12", "RATIO_quote_asset_volume_and_MA24", "RATIO_quote_asset_volume_and_MA48", "RATIO_quote_asset_volume_and_MA96", "RATIO_quote_asset_volume_and_MA192", "RATIO_quote_asset_volume_and_MA384", "RATIO_num_trades_and_MA3", "RATIO_num_trades_and_MA6", "RATIO_num_trades_and_MA12", "RATIO_num_trades_and_MA24", "RATIO_num_trades_and_MA48", "RATIO_num_trades_and_MA96", "RATIO_num_trades_and_MA192", "RATIO_num_trades_and_MA384", "RATIO_close_and_STD3", "RATIO_close_and_STD6", "RATIO_close_and_STD12", "RATIO_close_and_STD24", "RATIO_close_and_STD48", "RATIO_close_and_STD96", "RATIO_close

In [83]:
sql = sql + f" FROM trading_bot_historydataeth \
        WHERE timestamp >= '{result_str}' \
        ORDER BY timestamp;"

In [84]:
with conn.cursor() as cursor:
    cursor.execute(sql)
    result = cursor.fetchall()
conn.close()

In [85]:
df_ready_for_pred = pd.DataFrame(result, columns=columns)

In [86]:
df_ready_for_pred.head()

,id,hour_sin,hour_cos,day_sin,day_cos,mon_sin,mon_cos,weekday_sin,weekday_cos,year,...,RSI_384_quote_asset_volume,RSI_3_num_trades,RSI_6_num_trades,RSI_12_num_trades,RSI_24_num_trades,RSI_48_num_trades,RSI_96_num_trades,RSI_192_num_trades,RSI_384_num_trades,predicted
0,38190,1.000000,6.123234e-17,0.951057,-0.309017,0.5,0.866025,0.781831,0.62349,2024,...,49.859221,71.389892,3.699243,27.179893,46.358171,50.568465,49.547127,50.109445,49.851729,W
1,38191,0.965926,-2.588190e-01,0.951057,-0.309017,0.5,0.866025,0.781831,0.62349,2024,...,49.954691,3.279849,7.395918,30.689324,45.776484,49.810307,49.289525,49.963152,49.915666,W
2,38879,0.866025,-5.000000e-01,0.951057,-0.309017,0.5,0.866025,0.781831,0.62349,2024,...,50.052924,67.893374,68.134645,46.048634,49.646220,50.552824,50.354181,50.269552,50.092061,W
3,38880,0.707107,-7.071068e-01,0.951057,-0.309017,0.5,0.866025,0.781831,0.62349,2024,...,50.101788,73.389622,73.244999,52.088494,50.497962,51.041344,49.882164,50.246861,50.124498,W
4,38881,0.500000,-8.660254e-01,0.951057,-0.309017,0.5,0.866025,0.781831,0.62349,2024,...,49.864236,65.633652,52.506210,51.607839,49.419907,50.660680,49.553705,50.089724,49.785607,W


In [87]:
df_ready_for_pred.shape

(757, 67)

In [88]:
df_ready_for_pred[['predicted', 'id']].groupby('predicted').count()

,id
predicted,
B,6
H,163
S,32
W,556


In [89]:
tensor = dataframe_to_tensor(df_ready_for_pred.drop (columns= ['id', 'predicted']), TIME_STEP)

In [90]:
tensor.shape

(746, 12, 65)

In [91]:
predictions = model.predict(df_ready_for_pred.drop (columns= ['id', 'predicted']))

24/24 [==============================] - 0s 3ms/step


In [92]:
predictions.shape

(757, 4)

In [93]:
predictions_df = pd.DataFrame(predictions, columns=['B', 'H', 'S', 'W'])

In [94]:
predictions_df['Final'] = predictions_df.idxmax(axis=1)

In [95]:
predictions_df.head()

,B,H,S,W,Final
0,0.104594,0.084840,0.001344,0.809222,W
1,0.107970,0.085518,0.001027,0.805484,W
2,0.200521,0.054760,0.001701,0.743017,W
3,0.162601,0.076860,0.003634,0.756905,W
4,0.104117,0.065817,0.002506,0.827560,W


In [96]:
df_ready_for_pred['predicted'] = predictions_df['Final']

In [97]:
conn = db_connect.connect(CONFIG)

Connected to the PostgreSQL server.


In [98]:
for index, row in df_ready_for_pred.iterrows():
    sql = f"UPDATE trading_bot_historydataeth \
            SET predicted = '{row['predicted']}' \
            WHERE id = {row['id']};"
    with conn.cursor() as cursor:
        cursor.execute(sql)
    conn.commit()
conn.close()

In [99]:
conn = db_connect.connect(CONFIG)
sql = 'SELECT * FROM trading_bot_historydataeth \
        WHERE predicted is not null \
        ORDER BY timestamp ASC;'
with conn.cursor() as cursor:
    cursor.execute(sql)
    result = cursor.fetchall()
conn.close()

Connected to the PostgreSQL server.


In [100]:
conn = db_connect.connect(CONFIG)
sql = "SELECT COLUMN_NAME \
    FROM INFORMATION_SCHEMA.COLUMNS \
    WHERE TABLE_NAME = 'trading_bot_historydataeth' \
    ORDER BY ORDINAL_POSITION;"
with conn.cursor() as cursor:
    cursor.execute(sql)
    col_names = cursor.fetchall()
conn.close()

Connected to the PostgreSQL server.


In [101]:
cols_list = []
for col in col_names:
    cols_list.append(col[0])

In [102]:
result_df = pd.DataFrame(result, columns=cols_list)

In [103]:
result_df

,id,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,num_trades,...,RSI_384_num_trades,RSI_384_quote_asset_volume,RSI_3_num_trades,RSI_3_quote_asset_volume,RSI_48_num_trades,RSI_48_quote_asset_volume,RSI_6_num_trades,RSI_6_quote_asset_volume,RSI_96_num_trades,RSI_96_quote_asset_volume
0,38190,2024-01-09 06:00:00+05:00,2308.77,2309.08,2297.26,2302.00,8630.9460,1704780000000,1.988220e+07,21123,...,49.851729,49.859221,71.389892,24.804571,50.568465,50.447134,3.699243,1.500659,49.547127,49.153654
1,38191,2024-01-09 07:00:00+05:00,2302.00,2310.64,2298.58,2306.60,6331.7233,1704790000000,1.459177e+07,13559,...,49.915666,49.954691,3.279849,12.074540,49.810307,50.141823,7.395918,2.836300,49.289525,49.009751
2,38879,2024-01-09 08:00:00+05:00,2306.05,2311.30,2290.26,2291.28,11885.1589,1704790799999,2.734850e+07,29276,...,50.092061,50.052924,67.893374,66.430807,50.552824,50.582136,68.134645,61.900251,50.354181,49.723793
3,38880,2024-01-09 09:00:00+05:00,2291.28,2297.22,2285.40,2291.37,14051.9445,1704794399999,3.220718e+07,34420,...,50.124498,50.101788,73.389622,76.903579,51.041344,50.745154,73.244999,69.880417,49.882164,48.929447
4,38881,2024-01-09 10:00:00+05:00,2291.37,2301.73,2289.00,2292.40,9995.6604,1704797999999,2.294682e+07,23497,...,49.785607,49.864236,65.633652,65.543832,50.660680,50.580271,52.506210,54.083916,49.553705,49.579627
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
752,39631,2024-02-09 14:00:00+05:00,2513.27,2519.66,2481.47,2494.51,50546.2326,1707490799999,1.262731e+08,96795,...,50.657069,50.764413,60.610369,58.757069,53.100212,53.957455,65.450251,70.449525,52.741998,53.299243
753,39632,2024-02-09 15:00:00+05:00,2494.51,2501.46,2476.57,2495.33,31964.6794,1707494399999,7.956063e+07,65172,...,50.369280,50.504011,40.293519,42.167346,51.371620,52.489642,55.196642,58.655994,50.317666,51.117829
754,39633,2024-02-09 16:00:00+05:00,2495.33,2502.72,2478.07,2486.77,20726.9450,1707497999999,5.161690e+07,47178,...,50.231440,50.230337,48.720519,48.936317,49.285496,49.767510,53.449911,51.816573,49.934885,50.345761
755,39634,2024-02-09 17:00:00+05:00,2486.77,2500.87,2484.66,2497.19,18580.8317,1707501599999,4.637484e+07,42302,...,50.226517,50.235723,0.000000,0.000000,46.783647,46.406168,38.816488,35.807077,50.680708,51.051760


In [104]:
plotting.chart_with_markers(result_df, 'predicted')

c:\Users\user\Documents\GitHub\BinanceTradingBot\plotting.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\user\Documents\GitHub\BinanceTradingBot\plotting.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

